In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import folium
import requests
from folium.plugins import HeatMap

In [3]:
df = pd.read_csv("C:/Users/Mainaki Saraf/Desktop/Sem 5/DA/Project/Dataset/Sample_data_2017.csv")

In [4]:
df.columns

Index(['Unnamed: 0', 'Summons Number', 'Plate ID', 'Registration State',
       'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type',
       'Vehicle Make', 'Issuing Agency', 'Street Code1', 'Street Code2',
       'Street Code3', 'Vehicle Expiration Date', 'Violation Location',
       'Violation Precinct', 'Issuer Precinct', 'Issuer Code',
       'Issuer Command', 'Issuer Squad', 'Violation Time',
       'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation'],
      dt

In [5]:
df.iloc[:,25:30].head()
df.isna().sum()
#separate street dataset
df_street = pd.DataFrame(df['Street Name'])
print(df_street.shape)
#separate precinct data
df_precinct = pd.DataFrame(df['Violation Precinct'])
print(df_precinct.shape)
# drop street names which are NAN
df_street.dropna(inplace=True)
df_precinct = df_precinct.drop(df_precinct[df_precinct['Violation Precinct']==0].index)
print(df_street.shape)
print(df_precinct.shape)

(1080303, 1)
(1080303, 1)
(1079918, 1)
(873266, 1)


In [5]:
#function to get latitude and logitude based on precinct using Geocoding API

def get_lat_lon_pre(precinct):
    print(precinct)
    response = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address="+str(precinct)+"th precinct,NY"+"&key=INSERT_API_KEY")
    #print(response.json())
    lat = response.json().get('results')[0].get('geometry').get('location').get('lat')
    lon = response.json().get('results')[0].get('geometry').get('location').get('lng')
    return(lat,lon)

In [6]:
#call get_lat_lon for precinct
precinct_no = set(df_precinct['Violation Precinct'])
precinct_coords = dict()
for i in precinct_no:
    precinct_coords[i] = get_lat_lon_pre(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
39
40
41
42
43
44
45
46
47
48
49
50
51
52
55
56
57
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
94
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
632
120
121
122
123
127
130
144
161
163
202
803
805
806
301
824
317
878
415


In [7]:
#add lat lon to precinct dataframe
precinct_lat = dict()
for i in precinct_coords:
    precinct_lat[i]=precinct_coords[i][0]
precinct_lon = dict()
for i in precinct_coords:
    precinct_lon[i]=precinct_coords[i][1]

In [8]:
df_precinct['Latitude']= df_precinct['Violation Precinct'].map(precinct_lat)
df_precinct['Longitude']= df_precinct['Violation Precinct'].map(precinct_lon)

In [9]:
df_precinct['count']=1
print(df_precinct.head())
df_precinct.to_csv("precinct.csv")

   Violation Precinct   Latitude  Longitude  count
0                 121  40.623679 -74.150742      1
1                   1  40.720369 -74.006969      1
2                  46  40.853990 -73.900111      1
4                  13  40.736916 -73.982830      1
7                  20  40.784181 -73.974986      1


In [18]:
#function to get latitude and logitude based on Street Name using Geocoding API
def get_lat_lon_street(street_name):
    print(street_name)
    response = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address="+street_name+",NY"+"&key=INSERT_API_KEY")
    #print(response.json())
    lat = response.json().get('results')[0].get('geometry').get('location').get('lat')
    lon = response.json().get('results')[0].get('geometry').get('location').get('lng')
    print(lat,lon)
    return(lat,lon)

In [19]:
all_street_names = list(set(df_street['Street Name']))
len(all_street_names)

26227

In [20]:
#call get_lat_lon for street name first 5000
street_names = all_street_names[0:5000]
street_coords = dict()
ctr=0
for i in street_names:
    ctr+=1
    print(ctr)
    try:
        street_coords[i] = get_lat_lon_street(i)
    except:
        street_coords[i] = "not valid"

1
W 145 ST
40.8242227 -73.94519079999999
2
51TH AVE
40.7356414 -73.9046012
3
C/O JUNIPER BLVD
40.7188014 -73.8813763
4
QUEENS MIDTOWN EXPWY
40.7326614 -73.9199739
5
TIEMANN PLACE
40.8154454 -73.9594825
6
ROCKAWAY BLV D
40.6743114 -73.80136929999999
7
N/W/C NASSAU
42.51591450000001 -73.61011590000001
8
211 STREET
40.7550396 -73.7699487
9
NORMAN ST
40.6976066 -73.9007989
10
C/O ARCHER ST
40.64767 -73.588477
11
186TH AVE
40.8528978 -73.9327454
12
N/S 30 AV
40.75965860000001 -73.9012532
13
E 20 STREET
40.7360191 -73.9822744
14
Asch Loop
40.8710323 -73.8282906
15
Hallister St
40.5259881 -74.2186009
16
Isham St
40.8679271 -73.9196287
17
WEST 36
40.75347259999999 -73.9925244
18
72 DR
40.720313 -73.837957
19
N/E C/O AVE S
40.7410236 -73.9917641
20
W/S GUY R BREWER BLV
40.6809146 -73.7810444
21
W 15 STREET
40.7399454 -74.0008424
22
N/W C/O E 8TH ST
40.7256804 -73.9807682
23
EAST 227TH ST
40.888303 -73.856808
24
FORSYTH ST
40.7188482 -73.9925927
25
Reno Ave
40.5665378 -74.11879429999999
26
SB AM

40.649114 -73.9464826
200
CADMAN PLAZA
40.6976723 -73.9906433
201
N/S W 179 ST
40.7105864 -73.7835607
202
W 233 ST
40.8804433 -73.9030502
203
NEPTUNE AVE
40.5792604 -73.9819541
204
MOSHOLU GLF CO
40.8864981 -73.8826339
205
WEST BRIGHTON
40.6270298 -74.10931409999999
206
GRAHAM ST
40.7123777 -73.944034
207
SKILLMAN AVENUE
40.7472622 -73.9283221
208
9 MERTOTECH CENTER
40.6943147 -73.9842832
209
E/S HICKS
40.7127753 -74.0059728
210
N/O E 125TH ST
40.8107382 -73.95246329999999
211
CALLAOWAY ST
40.737499 -73.8565964
212
ALMEDA AVE
40.5971598 -73.7936811
213
BRITTON ST
40.8666101 -73.8691575
214
62nd Dr
40.7343236 -73.8554623
215
McKeever Pl
40.6646272 -73.95917570000002
216
S/S 56TH RD
40.7252801 -73.9164455
217
BRIGHTON BEACH LOT
40.5753524 -73.963831
218
GUNHILL RD
40.8734473 -73.8540762
219
E 97th St
40.7861708 -73.9495271
220
Avenue H
40.6317572 -73.9460498
221
Avenue U
40.600369 -73.94248499999999
222
W 77TH
40.7814934 -73.97845199999999
223
R/O 1575
40.7127753 -74.0059728
224
MORRIS A

40.5616379 -73.8953425
401
BRUCKNER BLVD (E/B)
40.8335819 -73.8281908
402
L/S W 6TH ST
40.7482164 -73.9893159
403
40th Rd
40.7571997 -73.8338989
404
E/B 10TH ST
40.7324394 -73.9935611
405
E 44ST
40.7524042 -73.9742911
406
122ND STREET
40.808865 -73.954048
407
Beach 106th St
40.582913 -73.82785539999999
408
C/O NARROWS
40.6066162 -74.0446602
409
WB E23 ST. 1 AV - LE
40.7368909 -73.97854029999999
410
99 STREET
40.7869726 -73.9475513
411
WEST 201TH ST
40.860201 -73.9205379
412
FMCP TENNIS COURTS
40.8248451 -73.93561170000001
413
Van Buren St
40.6910239 -73.9348596
414
BROOKHAVEN AVE
40.60004319999999 -73.75737529999999
415
MESEMAN AVE
41.2595797 -73.7541396
416
CATAMARAN WAY
40.5884383 -73.799291
417
E 233
40.8917815 -73.85227809999999
418
Wainwright Ct
40.582029 -73.8324702
419
Nixon Ct
40.5850218 -73.9677665
420
NB 65 ST. 13 AV - 11
40.625985 -74.0015109
421
HIGHLAND PLACE
40.6813492 -73.8811681
422
LIBERTY STREET
40.7097616 -74.0118569
423
BAY 22
40.6029293 -74.0023808
424
WICKHAM
40.8

40.8364591 -73.8608819
600
ACADEMY STREET
40.8634216 -73.9231399
601
SOUTH ELLIOT PLACE
40.6867708 -73.97547879999999
602
60 LA
40.763807 -73.97058899999999
603
ISERNIA AVE
40.5621959 -74.1108213
604
C/O 41 RD
42.54959909999999 -76.2888163
605
Paul Ave
40.8757296 -73.8906331
606
E/S MORRIS AVE
40.8414282 -73.9094697
607
EASTERN PARKWAY
40.6693118 -73.9366428
608
Purves St
40.7474275 -73.94144159999999
609
W 3
40.57669629999999 -73.9714905
610
WEST 40 ST
40.7564027 -73.9918198
611
EAST 221ST ST
40.8831409 -73.8562288
612
WALNUT AVE
40.8446354 -73.0370318
613
NORTHER BLVD
40.76539950000001 -73.8175923
614
C/O 20
40.7120212 -74.00949299999999
615
LYVERE ST
40.8417454 -73.85220939999999
616
Victor St
40.8455459 -73.8670721
617
CRAFT AVE
40.65065269999999 -73.7321498
618
BRIGHTON BECH
40.5780706 -73.9596565
619
W 96th St
40.7932251 -73.96837359999999
620
EAST 5TH
40.7248612 -73.98442
621
WB 69 AV 199 ST - 19
40.7360394 -73.7735381
622
DASH PL
40.888471 -73.90383369999999
623
149th St
40.814

40.6986946 -73.8009005
796
VAN WYCK T 1 JFK
40.6438733 -73.79068529999999
797
169th St
40.8402731 -73.92739619999999
798
SOXFORD ST
40.6858929 -73.9733657
799
57TH ST
40.767267 -73.9841435
800
FRANICS LEWIS BLVD
40.7241358 -73.7639272
801
E 60TH ST
40.7619245 -73.9655044
802
JFK TERMINAL 8 ARRIV
40.6413111 -73.77813909999999
803
GREENWICH AVENUE
40.736584 -74.0011012
804
C/O 95 NORTHERN BLVD
40.7571516 -73.8724354
805
W 122nd St
40.808865 -73.954048
806
DEAN AV
40.8342839 -73.8175321
807
N/E C/O 165 STREET
40.7410236 -73.9917641
808
DAVIDSON AV
40.8567706 -73.90605219999999
809
Tillary St
40.6961605 -73.98586650000001
810
Giegerich Pl
40.81427499999999 -73.803108
811
WEST 117 STREET
40.8038725 -73.9519366
812
C/O W 134 ST
40.7458339 -73.831304
813
E/S 1ST AVE
40.753843 -73.99581669999999
814
DELEY AVE
43.2994285 -74.21793260000001
815
NB 1 AV E102 ST. - E
40.7870698 -73.9419167
816
EAST 125 ST
40.8040915 -73.9367343
817
AVE S
40.6030108 -73.9615456
818
LINDEN BL
40.6579899 -73.8961789


...

40.76295160000001 -73.9692965
4073
93 AVE
40.7198949 -73.743666
4074
Yates Ave
40.8600948 -73.8549367
4075
LGA TERMINAL B AMERI
40.7746067 -73.8718579
4076
94th Ave
40.6975757 -73.81156419999999
4077
N/W CRNR 7 AVE
40.741844 -73.997613
4078
LENDON BLVD
40.6579899 -73.8961789
4079
Melyn Pl
40.6264269 -74.150854
4080
98th St
40.7878021 -73.95146059999999
4081
COMMERCE AVE
40.836079 -73.8402244
4082
78TH RD
40.7202568 -73.8149645
4083
FORREST AVE
40.813224 -72.87368599999999
4084
83rd St
40.7854763 -73.975821
4085
WEST 147 STREET
40.8253061 -73.9437651
4086
BEACH FRONT RD
40.5879878 -73.7948064
4087
Beaumont Ave
40.8535216 -73.8847886
4088
Brewster St
40.63255669999999 -74.0779598
4089
Bathgate Ave
40.8498932 -73.8954744
4090
E Buchanan St
40.6437685 -74.090591
4091
KINGS COLLEGE PL
40.8808102 -73.875647
4092
Mallow St
40.5466675 -74.2088582
4093
A  ST ANNS AVE
40.8123198 -73.9141048
4094
Hurlbert St
40.5921567 -74.0844716
4095
3 AVEE
40.8327501 -73.9050863
4096
S/S 42
41.8082532 -74.5565

40.834672 -73.939257
4264
FINDLAY AVENUE
40.8326318 -73.9124416
4265
MOSHOLU PARK WEST S
40.8821418 -73.886189
4266
B 97 STREET
40.79568829999999 -73.9716286
4267
E 120 STREET
40.8005138 -73.9381432
4268
BRYANT AVE
40.824723 -73.8886833
4269
R/S ROCKAWAY
40.6746338 -73.8061297
4270
S/S EGAN ST
40.6577607 -73.8705378
4271
CROSS BRONX EXPY
40.8331644 -73.86046449999999
4272
WEST 152ND  ST
40.8287195 -73.9421796
4273
R/S 156TH MORRIS
42.5486871 -75.24516849999999
4274
60th Dr
40.71693399999999 -73.8982255
4275
112TH ST
40.8023039 -73.9582698
4276
THOMPSON
41.6543833 -74.66346399999999
4277
Forbell St
40.6725492 -73.8626175
4278
S/S CHARLTON STREET
40.7272252 -74.0062602
4279
C/O B 95
40.7663693 -74.0670749
4280
ALBION AVE
40.7391928 -73.8847413
4281
C/O N 3 ST
40.7276592 -73.97625
4282
W 143RD ST
40.8219694 -73.9438541
4283
Demopolis Ave
40.549191 -74.161168
4284
Geranium Ave
40.7506113 -73.82124689999999
4285
153 ST
40.8297567 -73.9426914
4286
Clarkson Ave
40.6561084 -73.939568
4287
E 32

40.7079826 -74.0136184
4458
C/O 17TH AVE
40.622435 -73.990105
4459
E Scranton Ave
40.5545811 -74.15937199999999
4460
DEXTER CT
40.6925435 -73.86786029999999
4461
MORTEN ST
40.73095199999999 -74.0067226
4462
PIER 15
40.7050356 -74.0037533
4463
EAST 29 ST
40.7425974 -73.9803981
4464
N/W C/O MULBERRY ST
40.7189484 -73.9973819
4465
WEST 152 ST
40.8287195 -73.9421796
4466
CAMP ROAD
40.5972432 -73.7588971
4467
S/S OF MT. EDEN AVE
40.8431083 -73.9104298
4468
WEST 218TH ST
40.8716065 -73.9150554
4469
C/O WEBB AVE
40.8734449 -73.900958
4470
C/O 64 CIRCLE
40.74016049999999 -73.78029550000001
4471
CROWN ST
40.6658903 -73.94340799999999
4472
W 11TH ST
40.7363175 -74.0026576
4473
Sharrotts Rd
40.5372165 -74.2306062
4474
S/W C/O OF WYTHE AV
40.7218887 -73.9580971
4475
2ND STR
40.7641964 -73.9617361
4476
MARTIN L KING BLVD
40.8480135 -73.9186542
4477
WEST THAMES ST
40.7081035 -74.01666879999999
4478
DIVISON ST
40.714295 -73.99398339999999
4479
218th Pl
40.7259193 -73.7429622
4480
PARSON BL
40.7216453

40.8277802 -73.9437993
4648
E 60th Pl
40.6111885 -73.9112361
4649
ROOSEVELT ST
40.60589960000001 -74.09196399999999
4650
E 82 STREET
40.77547819999999 -73.9540605
4651
Tinton Ave
40.8202973 -73.9041123
4652
Coursen Pl
40.6194852 -74.079925
4653
159th St
40.8342199 -73.9413886
4654
JFK AIRPORT GREEN LO
40.6640109 -73.8200638
4655
Markham Pl
40.6154766 -74.13203519999999
4656
WYCOFF AVENUE
40.7003741 -73.9125838
4657
111th St
40.750231 -73.85513379999999
4658
93 RD
40.783266 -73.9507404
4659
E 36 STREET
40.7471251 -73.9774091
4660
ORANGE ST
40.69918699999999 -73.99398
4661
LGA LOT 5A
40.7769271 -73.8739659
4662
58 AVENUE
40.7261688 -73.8861234
4663
EAST 33 STREET
40.7455581 -73.97980319999999
4664
LINDEN BLVD (W/B) @
40.6549456 -73.916885
4665
E. Williston Avenue
40.7358606 -73.70679179999999
4666
S/W/C PARSONS BLVD
40.772356 -73.8246832
4667
W 59th St
40.7714206 -73.9899832
4668
W/S ADAMS ST
40.698303 -73.988849
4669
E/S 94TH ST
40.78433210000001 -73.95131860000001
4670
F/O 252 FRONT ST

40.8993283 -73.8639133
4842
S/W W 2 ST
40.7230145 -73.9857665
4843
C/O 76TH STREET
42.9489423 -88.00484
4844
C/O CYRPESS AVE
40.8013599 -73.916248
4845
Crooke Ave
40.652458 -73.9634282
4846
C/O E12 ST
40.7329026 -73.9889759
4847
Naughton Ave
40.5813022 -74.0890762
4848
KEARNEY AVE
40.8394217 -73.8245061
4849
E/S PRESCOTT PL
40.67752429999999 -73.9208943
4850
Alaska St
40.6365817 -74.1206697
4851
C/O 63 DR
40.7292664 -73.8630946
4852
Campbell Dr
40.839269 -73.8173973
4853
LEONARD STREET
40.7174249 -74.0052408
4854
169 STREET
40.8402731 -73.92739619999999
4855
E 10th St
40.7287833 -73.9844341
4856
MACY PL
40.8185164 -73.9020598
4857
E 3TH ST
40.7236548 -73.9852944
4858
JOHNSON AV
40.8818067 -73.91116199999999
4859
E MOSHOLU PK
40.873893 -73.8833964
4860
Plymouth St
40.7037027 -73.9846293
4861
MURRAY ST
40.7150284 -74.0120724
4862
E 219TH ST
40.8819226 -73.8575903
4863
N/W C/O RIVERSIDE DR
40.8136441 -73.9612685
4864
NORTH SERVICE RD
40.67244 -73.9648902
4865
AMETHYST STREET
40.8468156 -7

In [21]:
#add lat lon to precinct dataframe first 5000
street_lat = dict()
for i in street_coords:
    street_lat[i]=street_coords[i][0]
street_lon = dict()
for i in street_coords:
    street_lon[i]=street_coords[i][1]

In [22]:
#call get_lat_lon for street name 5000-10k
street_names_2 = all_street_names[5000:10000]
street_coords_2 = dict()
ctr=0
for i in street_names_2:
    ctr+=1
    print(ctr)
    try:
        street_coords_2[i] = get_lat_lon_street(i)
    except:
        street_coords_2[i] = "not valid"

1
Clay St
40.7366578 -73.954515
2
C/O 88TH
40.7644268 -73.9602652
3
ARLINGTON AVE
40.9008066 -73.9083891
4
N/S 53RD ST
40.7641748 -73.9847444
5
W 154
40.8289977 -73.9389727
6
WOOSTER STREET
40.7243186 -74.0010147
7
67th Pl
40.7032212 -73.88623539999999
8
56 PL
40.7611213 -73.9047879
9
KINGS HIGH
40.8833043 -73.2410912
10
C/O W/B EVELYN PL
40.60927969999999 -74.0648361
11
W/S CALLOWAY ST
40.737499 -73.8565964
12
WEST 185TH ST
40.8523077 -73.9334456
13
N/S HERKIMER ST
40.6784301 -73.9274257
14
PARSIFAL PL
40.8393633 -73.82148529999999
15
S/O 84-04 PARK LANE
40.7112813 -73.8350033
16
BEACH 122 STREET
40.5781391 -73.842224
17
Bay 29th St
40.60070229999999 -73.9963605
18
CEDAR ST
40.7081276 -74.0098349
19
WEST 39 ST
40.7552425 -73.99096949999999
20
VIELE AV
40.8069882 -73.8834597
21
Trent St
40.5515729 -74.14018639999999
22
143 RD ST
40.8219694 -73.9438541
23
C/O E 85 ST
40.7759951 -73.9496655
24
WEST 202 ST
40.8608744 -73.9202099
25
N/E C/O E 39 STREET
40.7552425 -73.99096949999999
26
LORI

40.7591409 -73.9067113
201
MORGAN AVE
40.7160374 -73.93599990000001
202
Old Amboy Rd
40.5472619 -74.15694289999999
203
C/O HIMROD ST
40.702231 -73.917243
204
CHURCH AVENUE
40.6508471 -73.94859749999999
205
DOVER ST
40.7440962 -73.9817188
206
W 55ST
40.7660013 -73.9852045
207
NORTH MOORE
40.7199639 -74.0085852
208
B 81 ST
209
TARAS SHEVCHENKO PLA
40.72870289999999 -73.98981739999999
210
238th St
40.886714 -73.90348569999999
211
NB CROTONA AV. E187
40.8539259 -73.8836385
212
S/W/C GREENE
42.3292408 -75.7699147
213
TIBBETT AVENUE
40.8818898 -73.9073722
214
W/S PROSPECT AVE
40.7590166 -73.0289727
215
NASHVILLE BL
40.691492 -73.747326
216
W 213TH ST
40.8687054 -73.9171832
217
E 96TH
40.7853417 -73.9496916
218
C/O BRUNCKER BLVD
40.805225 -73.9121651
219
CASALE PL
40.7890329 -73.9538458
220
WEST 96 ST
40.7932251 -73.96837359999999
221
S/S FLATLANDS AVE
40.6355816 -73.9134979
222
E 216 ST
40.8803506 -73.8604688
223
C/O HAROT
43.2994285 -74.21793260000001
224
N/W C/O LINCOLN PL
40.6706918 -73.9

40.7388514 -73.98522559999999
402
E/O 31 ST
40.74457 -73.9818134
403
Jasmine Ave
40.7536473 -73.8130549
404
FORSYTHE ST
40.7188482 -73.9925927
405
EB JAMAICA AV 171 ST
40.7074313 -73.7886899
406
E/S/O YOUNG AVE
40.8652684 -73.8481219
407
QUEENS AVE
40.691688 -73.704993
408
N/E C/O W 176 ST
40.8448971 -73.933244
409
MARION AVE
40.8656223 -73.8895222
410
E 45 ST
40.7530453 -73.9738796
411
47TH ST
40.7607273 -73.9882141
412
Crotona Park
40.8400367 -73.8953489
413
C/O SHORE RD
40.6196986 -74.04002349999999
414
21ST AVE
40.6095149 -73.9895787
415
N/S EAST 174 STREET
40.8374083 -73.8877819
416
WEST 142ND ST
40.82146789999999 -73.944604
417
VANBRUNT ST
40.6794584 -74.0104214
418
BAXTON
40.731892 -73.984586
419
C/O PATTERSON AVE
40.5789866 -74.08456050000001
420
C/O 83 AVE
42.948213 -78.857383
421
HOMECREST AVE
40.59423109999999 -73.9583652
422
Murdock Ct
40.585724 -73.96793439999999
423
CONCORD
42.5598469 -78.72494689999999
424
ROCKAWAY BOULEVARD
40.6743114 -73.80136929999999
425
E 165
40.826

40.6432174 -73.90306
597
N/E/C W 176 ST
40.8453633 -73.8968336
598
N/W/C BETHUNE
40.7369705 -74.0082618
599
LOUIS NINE BLVD
40.8327269 -73.8928887
600
Hegeman Ave
40.6599257 -73.8916423
601
BEACH 45TH ST
40.5966266 -73.7766673
602
Desmond Ct
40.5908707 -73.9605508
603
R/O 22-01
40.7127753 -74.0059728
604
109TH ST
40.8023751 -73.96430889999999
605
E/S 154 ST
606
PORTAL STREET
40.6674702 -73.9246799
607
Colon St
40.5311153 -74.1869435
608
105 AVE
40.702313 -73.784775
609
50th Ave
40.7383309 -73.9272957
610
53RD AVE
40.7497354 -73.77470079999999
611
BAY 50TH ST
40.5867064 -73.98573379999999
612
AMSTERDAM  AVE
40.8134463 -73.9562105
613
Bay Ridge Ave
40.626361 -74.008299
614
FT WAHINGTON AVE
40.84722989999999 -73.9397819
615
58 STREET
40.76786870000001 -73.9834487
616
Windham Loop
40.5769972 -74.1630804
617
C/O 69 AVE
40.7223985 -73.8681838
618
TULIP CT
41.1172087 -74.0238901
619
Montague Ter
40.6956456 -73.99738479999999
620
C/O E 24TH ST
40.7460428 -73.99873339999999
621
E 91ST  STREET
4

40.7238966 -73.8651379
796
QUEENSBRIDGE BABY
40.7535741 -73.94594740000001
797
S/S E 41 ST
40.6448359 -73.9962564
798
EAST 181
40.8506262 -73.8913504
799
W/S WEST 207TH ST
40.8658136 -73.9199866
800
Shore Dr
40.834511 -73.8167194
801
E S/O AVE S
40.8283137 -73.93770769999999
802
KINGSTON
41.9270367 -73.9973608
803
STEINWAY STREET
40.7670158 -73.91246749999999
804
W/S ELIZABETH ST
40.720633 -73.9949718
805
DEBROSSES ST
40.7234427 -74.0095405
806
MEMPHIS AVE
40.6618144 -73.7346611
807
THWAITES PL
40.8588707 -73.8695246
808
W/S ALLENDALE ST
40.6964027 -73.8082459
809
BEACH 32
40.5945752 -73.76440629999999
810
108TH AVE
40.696565 -73.791247
811
KINGSTON ST
41.9270367 -73.9973608
812
N/S W 115TH ST
40.6857504 -73.8277288
813
Hancock St
40.6849007 -73.9294398
814
258th St
40.7378159 -73.7104398
815
SCHENECTADY AV
40.6479969 -73.9329114
816
SCHERMERHORN AV
40.6890351 -73.9862265
817
C/O LINDEN BL
40.6579899 -73.8961789
818
C/O BROOKLYN AV
40.677208 -73.94395300000001
819
Mount Eden Ave
40.843

...

40.7478955 -73.9858445
4060
RIVER AV
40.8278208 -73.9259438
4061
MELROSE
42.8416173 -73.62380809999999
4062
Park Pl
40.7134016 -74.00954829999999
4063
LAGUARDIA
40.7769271 -73.8739659
4064
W 99th St
40.79691860000001 -73.9711392
4065
Kensington Ave
40.5948147 -74.0762594
4066
S/W C/O 133 AVE
40.6690038 -73.8050031
4067
C/O 43 ST
40.7560912 -73.9845066
4068
S/S CHASE CT
40.6431605 -73.9179858
4069
HIGHLAND PARK LOT
40.6887278 -73.8862997
4070
May Pl
40.5401184 -74.1714593
4071
HOE AVE
40.829588 -73.8908862
4072
DELAWARE AE
42.63924799999999 -73.7796145
4073
E 38TH ST
40.7483675 -73.9765992
4074
W Houston St
40.72803529999999 -74.0030307
4075
Miles Ave
40.818662 -73.818111
4076
56TH S
40.7665982 -73.984647
4077
LABURNUM AVENUE
40.7519025 -73.81293509999999
4078
PANREE AVE
40.7127753 -74.0059728
4079
N/W C/O W 2 ST
40.7127753 -74.0059728
4080
N/W C/O B 102 ST
40.791484 -73.9520122
4081
LORRAIN ST
40.7889 -73.5136417
4082
N/W CORNER OF 177 AU
10.79268 106.67775
4083
EAST NEW YORK
40.659052

40.7002048 -73.9538913
4252
E 115 STREET
40.7971024 -73.94013860000001
4253
127th St
40.7845099 -73.8412494
4254
DEWEY AV3E
43.2196201 -77.6412722
4255
GREENWITCH S
40.7221373 -74.0097722
4256
Pinehurst Ave
40.8515263 -73.9391059
4257
JORDAN ST
40.7682996 -73.78961009999999
4258
Brighton Beach Ave
40.577555 -73.96142890000002
4259
MANHATTAN AVENU
40.802479 -73.9586849
4260
UNDDERHILL AVE
40.6770612 -73.9659877
4261
W 32 STREET
40.7484063 -73.9884498
4262
GOETHALS RD
40.6234058 -74.17276729999999
4263
E 52nd St
40.75700519999999 -73.96965999999999
4264
W 2 STREET
40.6085009 -73.9759425
4265
W 35TH ST
40.75285700000001 -73.9929789
4266
Thollen St
40.5558368 -74.1310699
4267
9 ST
40.7337631 -73.9981191
4268
C/O 94 ST
40.794903 -73.975785
4269
173 STREET
40.8446428 -73.93872
4270
PUTMAN AVE
42.9563313 -74.381817
4271
E/S E 57 ST
40.767267 -73.9841435
4272
C/O 26 ST
40.75008500000001 -74.004434
4273
E 64th St
40.7646896 -73.9643068
4274
New Utrecht Ave
40.6262341 -73.99697429999999
4275
E 5

40.728901 -73.7820311
4445
FORSYTH STREET
40.7188482 -73.9925927
4446
EAST 198TH STREET
40.8690282 -73.88905
4447
S/S E 25TH STREET
40.7464345 -73.99720239999999
4448
LORIMER
40.71477300000001 -73.94950419999999
4449
3RD
40.8327501 -73.9050863
4450
BAILEY PL
40.638782 -74.1693938
4451
WILLIAM STREET
40.7079971 -74.0079973
4452
Wade St
40.6158362 -74.14189979999999
4453
NEW KIRK AVE
40.6377583 -73.9587126
4454
LASALE ST
40.8133806 -73.9586113
4455
OCEAN PKY
40.61287799999999 -73.96838620000001
4456
S/S W 29 ST
40.7659885 -73.9243365
4457
2 AVENUE
40.7641964 -73.9617361
4458
EAST 77 STREET
40.7726397 -73.9574725
4459
Raymond Pl
40.6278886 -74.1243904
4460
WEBB AVENUE
40.8679558 -73.9029134
4461
SB ROCKAWAY AV LOTT
40.6656481 -73.9097069
4462
Caswell Ln
40.6180809 -74.1669165
4463
S/E C/O E 18TH
43.2994285 -74.21793260000001
4464
66 RD
40.7294412 -73.84852169999999
4465
124TH ST
40.8037062 -73.9379199
4466
BRIDGE ST
40.6976238 -73.9849705
4467
W/S E 55 ST
40.7660013 -73.9852045
4468
W/S G

40.77086389999999 -73.95777199999999
4638
EAST TRMONT
40.8453781 -73.8909693
4639
EAST 230TH STREET
40.8899312 -73.8541034
4640
SB WEST END AV W97 S
40.786963 -73.9793487
4641
SAINT NICHOLAS AVE
40.8280385 -73.9424401
4642
140 ST
40.8205972 -73.9464625
4643
EAST 3RD STREET
40.7236548 -73.9852944
4644
80TH ST
40.7744413 -73.9555424
4645
Ericsson St
40.7639432 -73.8657999
4646
PROVOST ST
40.7335083 -73.9496958
4647
W 31 ST
40.575208 -73.9974919
4648
EAST 10 STREET
40.7287833 -73.9844341
4649
W/S WALES AVE
40.8114057 -73.9077786
4650
MOUNT EDN PARKWAY
40.8427516 -73.9088806
4651
LICHFIELD RILA PARKI
52.681602 -1.831672
4652
Malcolm X Blvd
40.8096568 -73.9442007
4653
E 237
40.8993283 -73.8639133
4654
Meadow Ave
40.7119553 -73.5567769
4655
LITTLE W 12 STREET
40.7403074 -74.00795839999999
4656
E/N C/O E 182 ST
35.7595731 -79.01929969999999
4657
WEST 57
40.767267 -73.9841435
4658
EAST HOUSTON
40.7221373 -73.985796
4659
E 28TH  STREET
40.7422936 -73.9816311
4660
S/E C/O MACON ST
40.6598677 -73

40.786963 -73.9793487
4831
W 89TH ST
40.7894734 -73.97325769999999
4832
C/O BAYVIEW
40.9057261 -73.54897609999999
4833
S/S E 168TH STREET
40.8394861 -73.9360329
4834
NB WEST END AV W81 S
40.7856469 -73.9801636
4835
PIER 15 OPPOSITE OF
40.7050356 -74.0037533
4836
C/O W 54
40.89939950000001 -73.3528168
4837
N/E 12TH AVE
40.7300131 -73.9835216
4838
Van Pelt Ave
40.6313111 -74.1552777
4839
CASSIDY PL
40.6419568 -74.0958477
4840
W/S LOGAN ST 100
40.6843357 -73.87871729999999
4841
11TH AVE
40.7437681 -74.0086431
4842
HOLLAND AVE
40.86168809999999 -73.86551469999999
4843
SNEDIKER
40.6654761 -73.9000046
4844
HENDRIX ST
40.6594298 -73.8837346
4845
N/W/C/O 77 AVE
40.72260989999999 -73.81477699999999
4846
S/S E HOUSTON ST
40.7221373 -73.985796
4847
N/W C/O CRUGER AVE
40.8548247 -73.86644160000002
4848
E 219th St
40.8819226 -73.8575903
4849
Rockland Ave
40.5885534 -74.1446702
4850
I/O FULTON ST
40.7115663 -74.0114261
4851
HAMPTON PLACE
42.6908504 -73.679203
4852
John St
40.7082349 -74.0065387
4853

In [23]:
#add lat lon to precinct dataframe first 5000
street_lat_2 = dict()
for i in street_coords_2:
    street_lat_2[i]=street_coords_2[i][0]
street_lon_2 = dict()
for i in street_coords_2:
    street_lon_2[i]=street_coords_2[i][1]

In [24]:
#call get_lat_lon for street name 10k-15k
street_names_3 = all_street_names[10000:15000]
street_coords_3 = dict()
ctr=0
for i in street_names_3:
    ctr+=1
    print(ctr)
    try:
        street_coords_3[i] = get_lat_lon_street(i)
    except:
        street_coords_3[i] = "not valid"

1
S/E C/O E 181 ST
40.8503015 -73.9361431
2
C/O BARRY ST
40.700511 -73.803324
3
SB HYLAN BLVD @ FING
40.5464584 -74.1414633
4
TERMINAL B ARRIVALS
40.7741212 -73.8721218
5
E 139 ST
40.8075129 -73.91702409999999
6
N/E C/O 39 AVE
40.7599095 -73.8316067
7
147th Pl
40.698692 -73.805694
8
NEW ENGLAND THRWAY
40.8862382 -73.80814289999999
9
C/O W 206 ST
40.757592 -73.77665189999999
10
WB ASTORIA BV. 100 S
40.7614838 -73.8688751
11
147 DR
40.65587 -73.736679
12
Hawthorne Ave
42.6728043 -73.7984558
13
PIERREPONT PLACE
40.6962276 -73.99710859999999
14
WESTWOOD AVENUE
40.60557499999999 -74.13178169999999
15
Tudor St
40.5583097 -74.1465163
16
N/S SHEEPSHEAD BAY
40.5953955 -73.94575379999999
17
T-8 RED LOT
40.7127753 -74.0059728
18
E/S W 36 ST
40.7471251 -73.9774091
19
NEW HAVEN AVENUE
40.6004802 -73.75219729999999
20
E 229 DR SOUTH
40.8855212 -73.8447937
21
103RD RD
40.703935 -73.784409
22
PARSON BLVD
40.772356 -73.8246832
23
DRESIER LOOP
40.8778736 -73.8295294
24
W/S/O BANKS ST
40.8055791 -73.9666

40.7453458 -73.981183
202
ST NICHOLAS STREET
40.8280385 -73.9424401
203
EAST 30TH STREET
40.7432078 -73.9799487
204
MARLBOURGH ROAD
40.6412196 -73.9644311
205
FMCP TENNIS COURST
40.7480979 -73.8097785
206
SB UTICA AVE @ AVE N
40.61895639999999 -73.92694019999999
207
N/E C/O 85 ST
40.7800693 -73.9597685
208
SANDERS  ST
40.6240574 -74.1518964
209
W/S CONEY ISLAND
40.5755438 -73.9707016
210
C/O EVERGREEN
40.7372311 -73.8763527
211
20 AV
40.612004 -73.99055899999999
212
W/S OF UNIONPORT RD
40.8412549 -73.8634034
213
Metropolitan Ave
40.7123481 -73.8913982
214
EAST 141 ST
40.8089719 -73.9166019
215
W/S OF GRANT AVE
40.6830197 -73.8671366
216
SEDGWICK AVENUE
40.8603888 -73.91150259999999
217
GRENFELL ST
40.71038799999999 -73.8327629
218
WB JAMAICA AVE @ 224
40.7215302 -73.7310298
219
EVELYN PLACE
40.60927969999999 -74.0648361
220
W 15th St
40.7399454 -74.0008424
221
EAST 3ST
40.7236548 -73.9852944
222
C/O BLAKE AVE
40.671194 -73.870049
223
FORT GEORGE HILL
40.8591853 -73.9265166
224
SETON AV

40.8192567 -73.92388079999999
400
LUDLOW AVE
40.71873129999999 -73.9890522
401
GARRETSON AVE
40.5889398 -74.09523329999999
402
PAERDEGAT ST
40.63121599999999 -73.9081929
403
FMCP AQUATIC CT
40.7517801 -73.8376576
404
Winchester Blvd
40.7324714 -73.73554659999999
405
BOYLAND AVE
40.6688509 -73.91347309999999
406
JUMER PL
41.458272 -90.61334599999999
407
B 63 STREET
40.5951646 -73.7925351
408
S/E NEPTUNE AVE
40.6633213 -73.3761114
409
WEST 33RD ST
40.7521095 -73.9953486
410
EB FULTN ST. HOWRD A
40.7127753 -74.0059728
411
N/W C/O DITMAS AVE
40.6366606 -73.9679811
412
SWEETGUM LANE
40.5920937 -74.1523383
413
E 18th St
40.735961 -73.9857653
414
W 225TH
40.8742793 -73.90924729999999
415
B 17 S TREET
43.2994285 -74.21793260000001
416
S/S 90 AVE
40.7266361 -73.72135039999999
417
Kathleen Ct
40.508658 -74.2346417
418
W 67th St
40.7742367 -73.9811285
419
20RD AVE
40.7816876 -73.82166810000001
420
THORNHILL AVE
40.7616101 -73.733775
421
MCKINBBIN ST
40.705342 -73.9389939
422
GLEENE ST
40.7237963 

40.5809995 -73.8304514
596
Harrison St
40.7188367 -74.0106441
597
Harper Ave
40.8882998 -73.8332231
598
W 240TH ST
40.8872777 -73.90238599999999
599
CARROLL PL
40.6473568 -74.0840504
600
N/S W 100
40.7127753 -74.0059728
601
WESTERLEIGH PARK
40.6197065 -74.13446669999999
602
NORTON
42.4114705 -74.0668009
603
N/W C/O W 179
40.7127753 -74.0059728
604
STONE
40.70403100000001 -74.0115842
605
BAY RIDGE AVE
40.626361 -74.008299
606
Vleigh Pl
40.723143 -73.8204787
607
QUEENS BLVD (E/B) @
40.73587260000001 -73.9939717
608
Mount Carmel Pl
40.7398575 -73.977751
609
E 223rd St
40.8837192 -73.8528149
610
MALCOLM X
40.8020033 -73.9501856
611
31 ST STREET
40.7688417 -73.9195183
612
RIVINGTON BLVD
40.7196031 -73.986758
613
116 RD
40.683183 -73.789292
614
GREENWOOD CT
40.6021093 -73.74788029999999
615
OCEAN PKWY
40.61287799999999 -73.96838620000001
616
CONTROL TERMINAL B
40.7769271 -73.8739659
617
135 STREET
40.81708450000001 -73.94791649999999
618
SULIVAN ST
40.7276175 -74.00143729999999
619
Blake Ct


40.7669796 -73.9839556
794
Richards St
40.6774177 -74.01011179999999
795
BALTIC
40.6842031 -73.990189
796
GREENWPOINT AVE
40.7322101 -73.957838
797
Rodman St
41.44773 -74.4048973
798
BROXWOOD AVE
40.8768689 -73.8597065
799
NB 4 AV. SACKETT ST-
40.678082 -73.9826769
800
59 ST COLUMBUS CIRCL
40.7687804 -73.9821154
801
S/S 130 AVE
40.6708034 -73.7958288
802
NB LUTEN AV HYLAN BV
40.5231708 -74.1885465
803
62 STREET
40.7630254 -73.9642329
804
170TH STREET
40.8420924 -73.9262317
805
Hart St
40.6971851 -73.9288241
806
BERGEN  STREET
40.6761 -73.9499161
807
BUHRE AVE
40.847193 -73.8317622
808
N/S QUEENS PLAZA
40.7509846 -73.9402762
809
35TH ST
40.75285700000001 -73.9929789
810
W/S CONEY ISLAND AV
40.6136033 -73.9631313
811
W/S ARTHUR AVE
40.851226 -73.89050739999999
812
RIDGEWOOD AVE
40.6840108 -73.8783555
813
Hampton Ave
40.579775 -73.94869290000001
814
EAST 23RD
40.7384996 -73.9823607
815
BLDG 67
40.77557059999999 -73.9833877
816
West Rd
43.6691242 -75.43564909999999
817
64th St
40.7646896 -

...

40.6253622 -74.1664504
4074
FITCHETT ST
40.7208135 -73.8645815
4075
196th St
40.7329623 -73.7759009
4076
MENAHAN STREET
40.7023598 -73.91287419999999
4077
LOWE CT
40.7027462 -73.8058904
4078
159 AVE
40.658769 -73.842709
4079
R/O 539 E FORDHAM
40.8595472 -73.886775
4080
33RD ROAD
40.77085710000001 -73.7751596
4081
BRAODWAY AVE
40.81241989999999 -73.96053429999999
4082
DELONG STREET
40.7547916 -73.83602789999999
4083
E 106 ST
40.7915312 -73.9447036
4084
SOUTHER BLVD
40.8374083 -73.8877819
4085
90 ST
40.7899188 -73.97238159999999
4086
WILLIAMSBRIDGE RRD
40.8550982 -73.85510719999999
4087
E/S OF 6TH AVE
40.742903 -73.99279779999999
4088
VICTORY BLVD (E/B) @
40.6143303 -74.10899289999999
4089
TIERNEY PLACE
40.8136903 -73.80266480000002
4090
11st St
40.7293824 -73.9840037
4091
W/O/N/W/C/O PENELOPE
40.743685 -73.981458
4092
TELLER AVE
40.8329929 -73.9111898
4093
E/S 18TH ST
40.735961 -73.9857653
4094
Warsoff Pl
40.6983482 -73.9540987
4095
Charleston Ave
40.5491633 -74.2176618
4096
LENOX RD
40

40.7527048 -73.9946703
4263
GREEN WICH ST
40.7221373 -74.0097722
4264
W 14 TH ST
40.7393254 -74.00148089999999
4265
N/E C/O MARION AVE
40.7985798 -73.94832640000001
4266
BOWNE
40.75552030000001 -73.8202866
4267
E 201 ST
40.8709322 -73.8834548
4268
N/E JEROME
40.8654302 -73.8989976
4269
W 63 ST
40.7729227 -73.9857785
4270
Champlain Ave
40.5614353 -74.12646509999999
4271
121 STREET
40.80797279999999 -73.95387
4272
STUY OVAL
40.7313441 -73.97805439999999
4273
W 124 ST
40.8082644 -73.9487541
4274
DEWEY AV
43.2196201 -77.6412722
4275
N/W C/O 86 DR
43.2994285 -74.21793260000001
4276
W 98th St
40.7962763 -73.97156810000001
4277
114th Dr
40.696653 -73.7617446
4278
Red Cross Pl
40.6994872 -73.9891699
4279
SCHERMERHORN STREET
40.6890351 -73.9862265
4280
C/O 7895
40.7127753 -74.0059728
4281
MIDDLETOWN
41.4459271 -74.42293409999999
4282
BEACH 72 STREET
40.5953639 -73.80093529999999
4283
ASTT AVE
40.7127753 -74.0059728
4284
SUTTON ST
40.7252712 -73.9406297
4285
RIVER
40.7802261 -73.90230079999999
4

40.7260554 -73.9835429
4455
Stratford Ave
40.8292403 -73.8755134
4456
E/S AUDUBON AVE
40.8478725 -73.9329806
4457
Vance St
40.8607738 -73.83182959999999
4458
OPP 9505 AVE D
40.6492872 -73.9090226
4459
C/O BEACH 80 ST
40.5899232 -73.8065156
4460
WARRING AVE
40.8610377 -73.8502203
4461
E/S PROSPECT
43.30423589999999 -75.1521082
4462
E 7TH
40.7260554 -73.9835429
4463
E/S WEBSTER AVENUE
40.8647524 -73.8875207
4464
E 116
40.7973605 -73.93860289999999
4465
E 105 ST
40.790896 -73.9451536
4466
CADMAN PLAZA W
40.6971049 -73.9913143
4467
HANSON PLACE
40.6853983 -73.9752374
4468
WEST 105TH STREET
40.7995653 -73.9657474
4469
DAVIT COURT
40.8064962 -73.8546447
4470
EVERGREEN
40.750444 -73.98198
4471
IMLAY STREET
40.6815002 -74.00919340000002
4472
HALL PL
40.8252799 -73.8975983
4473
8 AVENUE
40.8164218 -73.9466138
4474
CROWN STREET
40.6658903 -73.94340799999999
4475
WALTON AVENUE
40.8383387 -73.9169127
4476
Boscobel Pl
40.8442057 -73.92454959999999
4477
VERMILYEA AVENUE
40.8660651 -73.9224986
4478
G

40.6758821 -73.84499509999999
4646
W/S LEXINGTON AVE
41.0228349 -73.77061499999999
4647
Mac Donough St
40.6827034 -73.9290159
4648
I/O 3524 E 36
40.7471251 -73.9774091
4649
86TH STR.
40.7780602 -73.9521931
4650
MCDOANLD AVE
40.6224684 -73.9755705
4651
E 117 S T
40.79768019999999 -73.93728949999999
4652
E 11TH STREET
40.7293824 -73.9840037
4653
Wilson Ter
40.6043827 -74.0956372
4654
LORRAINE STREET
40.67370380000001 -74.0044427
4655
E 118th St
40.7986115 -73.9375225
4656
131ST ST
40.8139554 -73.94843490000001
4657
126 STREET
40.8107705 -73.95049829999999
4658
Montague St
40.6949747 -73.9949634
4659
Ebey Ln
40.5487368 -74.1971843
4660
E 53 ST
40.75771049999999 -73.9694269
4661
ROBERTS AVE
40.8456814 -73.8352582
4662
S/W PORTAL
40.7127753 -74.0059728
4663
GIEGERICH PL
40.81427499999999 -73.803108
4664
Atlantic Ave
40.676295 -73.9080267
4665
STRAUSS ST
40.663054 -73.91487029999999
4666
GALES PL
40.7899751 -73.9539066
4667
60TH STREET
40.7619245 -73.9655044
4668
Zulette Ave
40.8421393 -73.8

40.8089719 -73.9166019
4837
RIVINGTON
40.7196031 -73.986758
4838
ST.NICHOLAS
40.7041208 -73.9158559
4839
ESSEX DR
40.581406 -74.1584549
4840
GLEN STREET
40.6092295 -74.1801728
4841
Beach 15th St
40.5977028 -73.7490554
4842
WOLLNIAN RINK CENTRA
40.7677792 -73.9744673
4843
SEDGWICK
43.065697 -76.13177999999999
4844
I/O CAPODANNO BLVD
40.5811503 -74.0757714
4845
32nd Ave
40.7577699 -73.88503879999999
4846
Alter Ave
40.5899466 -74.09325849999999
4847
C/O PERRY
42.7041129 -78.0135801
4848
KERMIT AVE
40.5975056 -74.08305349999999
4849
WINDSOR PL
40.6594255 -73.9800884
4850
Lamberts Ln
40.6166899 -74.1684871
4851
S CONDUIT AVE
40.66564169999999 -73.793944
4852
W FORDHAM RD (W/B) @
40.8624921 -73.9028908
4853
WENTWORTH AVE
40.5916106 -74.0716499
4854
ST JOHN PLACE
40.6715288 -73.9475595
4855
73rd Ave
40.7379181 -73.7679589
4856
E 50
40.7558356 -73.970781
4857
N/E C/O BEAUMONT AVE
40.7985798 -73.94832640000001
4858
KINGSLAND AVENUE
40.7258892 -73.9416945
4859
E/S E 57TH ST
40.767267 -73.9841435

In [25]:
#add lat lon to precinct dataframe 10k-15k
street_lat_3 = dict()
for i in street_coords_3:
    street_lat_3[i]=street_coords_3[i][0]
street_lon_3 = dict()
for i in street_coords_3:
    street_lon_3[i]=street_coords_3[i][1]

In [26]:
#call get_lat_lon for street name 15k-20k
street_names_4 = all_street_names[15000:20000]
street_coords_4 = dict()
ctr=0
for i in street_names_4:
    ctr+=1
    print(ctr)
    try:
        street_coords_4[i] = get_lat_lon_street(i)
    except:
        street_coords_4[i] = "not valid"

1
Newkirk Ave
40.6377583 -73.9587126
2
Court Sq
40.74544909999999 -73.9433105
3
GERVIL STREET
40.5464944 -74.2118729
4
C/O DICKENS ST
40.755834 -73.574338
5
RIVINGTON ST
40.7196031 -73.986758
6
E/B E 178 ST
40.8457148 -73.8893973
7
W 19TH ST
40.7425781 -73.99965159999999
8
TENEYCK ST
40.7100162 -73.9456776
9
Bullard Ave
40.90184920000001 -73.85703749999999
10
C/O DITMAS AVE
40.6364343 -73.97855369999999
11
Houseman Ave
42.356138 -73.607604
12
GRACE AVENUE
40.8939384 -73.8459762
13
C/O RIVERDALE AVE
40.90680990000001 -73.9049026
14
BRAODWAY
40.81241989999999 -73.96053429999999
15
Greenwich Ave
40.736584 -74.0011012
16
C/O KINGS HWY
40.605962 -73.98789699999999
17
S/E C/O 37 AVE
40.7717998 -73.9089353
18
DEEMS AVE
40.6172395 -74.1322646
19
W 252 ST
40.8996108 -73.9088383
20
PITT
21
EAST 23 STREET
40.7384996 -73.9823607
22
YELLOW STONE BLVD
40.7225594 -73.852083
23
CROTONA PK NORTH
40.8413665 -73.8927009
24
S/E C/O 8TH AVE
40.7420845 -73.9998396
25
C/O WILLIAMSBRIDGE
40.8763053 -73.860755

40.6272858 -74.1041825
202
E 104ST
40.7909485 -73.94723789999999
203
W/S HOYT ST
40.6836302 -73.989791
204
S/S E 61 ST
40.7624726 -73.9648836
205
W Fingerboard Rd
40.6018573 -74.0816169
206
C/O MARLBOROUGH RD
40.635792 -73.96338659999999
207
I/O 242 BROADWAY
42.8113044 -73.94415649999999
208
S/W C/O 92 ST
40.7830623 -73.95268039999999
209
ESPLANADA AVE
40.8587144 -73.85595169999999
210
Tenney Pl
40.8483807 -73.91916619999999
211
C/O W 7 ST
40.7260554 -73.9835429
212
E 117TH ST
40.79768019999999 -73.93728949999999
213
E/S C/O OAKTREE PL
40.837582 -73.910145
214
WINCHESTER BLVD
40.7324714 -73.73554659999999
215
Wyckoff St
40.6852478 -73.98871559999999
216
S/W/C INDIAN RD
40.8723985 -73.9192758
217
S/W C/O SOUTH ST
40.7055641 -74.0017386
218
S/S WEST OF FOSTER A
40.7127753 -74.0059728
219
THOMPKIN AVE
40.6898391 -73.9452019
220
BERRIMAN STREET
40.6686523 -73.8784909
221
LELAND AVE
40.8259709 -73.86216619999999
222
AMADAR STREET
40.7127753 -74.0059728
223
135th Ave
40.6757563 -73.7448855
2

40.6835363 -73.783011
397
Chestnut Ave
40.6182104 -73.9610129
398
ETNA STREET
40.686506 -73.874951
399
E 13th St
40.7306517 -73.9830602
400
WTSON AVE
40.828049 -73.86357199999999
401
CELTIC AVE
40.7372005 -73.92242829999999
402
ST LANRENCE AVE
40.8328065 -73.8675382
403
HALLECK ST
40.8092968 -73.8801677
404
Havilan Ave
40.8303261 -73.85199089999999
405
WEST 19
40.7425781 -73.99965159999999
406
WHITTLOCK AVE
40.8252833 -73.8865873
407
N/E C/O WEST 88 STRE
40.7894384 -73.9744005
408
LALAND AVE
40.7515661 -74.00418719999999
409
S/E C/O QUARRY RD
40.7558722 -73.9701462
410
137 ST
40.8182444 -73.9466422
411
Coyle St
40.5886994 -73.93482709999999
412
WEST 129TH STREET
40.8129733 -73.94991569999999
413
117th St
40.79768019999999 -73.93728949999999
414
WALDO PL
40.6081969 -74.1170459
415
NB TARGEE ST. DEKALB
40.6127252 -74.0850018
416
W/S NEW JERSEY AV
40.6652053 -73.89313159999999
417
EB WOODRW RD. ELLSWR
40.7127753 -74.0059728
418
Rockville Ave
40.5957306 -74.1645655
419
N/S 45TH ST
40.75957

40.7189066 -73.7482213
593
AUBURN
42.9317335 -76.5660529
594
STAIN NICOLAS AVE
40.8280385 -73.9424401
595
EAST 86 STREET
40.7780602 -73.9521931
596
NB MAIN ST @ REEVES
40.7396271 -73.82467919999999
597
BAY 50 ST
40.5867064 -73.98573379999999
598
W 49
40.76203659999999 -73.9874564
599
SEVEN AVE
40.75177480000001 -73.9900696
600
R/O 159-64 HARLEM RI
40.79031699999999 -73.9537622
601
E 85 ST
40.7781416 -73.9544957
602
W 120TH ST
40.8072091 -73.95399820000002
603
Doty Ave
40.593135 -74.0648557
604
Whitestone Expy Svc
40.7821556 -73.82612739999999
605
HUDSON  ST
40.7408398 -74.00513819999999
606
Aberdeen St
40.6822035 -73.90579699999999
607
Delancey Pl
40.8470935 -73.86072159999999
608
MONTGOMERY PLACE
42.0145766 -73.9189633
609
SUYDAM PL
40.67767569999999 -73.92655979999999
610
Andrea Pl
40.6253622 -74.1664504
611
EAST 165TH STREET
40.8264916 -73.90768280000002
612
E 141ST STREET
40.8089719 -73.9166019
613
MILLER FIELD PKG LOT
40.5673593 -74.09903609999999
614
POWERS
41.4366966 -73.5733274

41.041276 -73.7716343
788
N 4th St
40.7169714 -73.9607879
789
NORTHSIDE OF BECK ST
40.8163963 -73.9004236
790
E 46 ST
40.7535458 -73.9730866
791
BRIGHTON 15 ST
40.5766023 -73.9547789
792
S/W/C W 27
43.2994285 -74.21793260000001
793
59 STREET
40.7661851 -73.9773038
794
C/O 162 ST
42.9512045 -76.5831604
795
E 57TH ST
40.7601787 -73.967331
796
N 4 ST
40.7169714 -73.9607879
797
CROTONA PARK PL
40.8400367 -73.8953489
798
EINSTEN LOOP
40.8652377 -73.8223883
799
N/E/C CLINTON
43.0484029 -75.3785034
800
EL CUMINO LOOP
40.5373175 -74.2137797
801
WEST 18
40.7417321 -73.9994562
802
Inwood St
40.681613 -73.796395
803
C/O 90 AVE
40.7505926 -73.97917699999999
804
C/O 125TH ST
40.814693 -73.95703999999999
805
NB FLATBUSH AVE @ AV
40.630641 -73.94535479999999
806
Ryerson St
40.6935602 -73.96499949999999
807
146 STREET
40.8246948 -73.9442246
808
GILLESPIE AVE
40.8409932 -73.8307544
809
C/O WARREN ST
40.7143096 -74.0076697
810
HARVEY PARK
40.7836138 -73.82328559999999
811
LASALLE ST PARKING L
40.7295837

...

40.7610623 -74.0000827
4063
TRANTOR PL
40.629135 -74.1444221
4064
ST NICHOALS AVE
40.8280385 -73.9424401
4065
EAST 126TH ST
40.8047954 -73.9363171
4066
74TH STREET
40.7705669 -73.9584414
4067
Devon Loop
40.5807778 -74.1604989
4068
PLAZA DRIVE
40.8400444 -73.91821780000001
4069
PIERREPORT ST
40.695463 -73.993978
4070
S/O 3050 PARK AVE
40.8223112 -73.9194918
4071
JFK LOT 9
40.6640109 -73.8200638
4072
PIER FIVE BROOKLYN B
40.6995399 -73.9973777
4073
S/W C/O BEDFORD AVE
40.6537806 -73.956165
4074
McDonald Ave
40.6224684 -73.9755705
4075
Aitken Pl
40.692435 -73.994463
4076
Mount Hope Pl
40.8486333 -73.9072152
4077
OVINGTON AVENUE
40.63030029999999 -74.0163464
4078
N/S MADISON AVE
40.7788867 -73.9602326
4079
OAKDALE ST
40.5424392 -74.16096759999999
4080
S/W C/O 19 ST
40.7455173 -74.0062681
4081
Prospect Park West
40.6644777 -73.97699829999999
4082
E 8
40.7256804 -73.9807682
4083
45 ROAD
40.75691339999999 -73.7791277
4084
Circle Dr
40.8107222 -73.69147939999999
4085
HONEYWELL ST
40.7494915 -7

40.6052323 -73.766741
4255
FERNDALE ST
42.6579037 -73.8127781
4256
Beach 57th St
40.5938043 -73.7871208
4257
HAVEMEYER
40.712239 -73.95713599999999
4258
E 54TH STREET
40.7582569 -73.96874609999999
4259
BAY 38 ST
40.594795 -73.9916966
4260
42RD ST
40.7578775 -73.99132759999999
4261
WICKHAM AV
40.8690765 -73.8373749
4262
N/W C/O 10TH AVE
40.759144 -73.995603
4263
GRAHAM
41.123707 -73.8101339
4264
VAN COURTLAND AVE
40.87681509999999 -73.8838804
4265
PIERREPONT
44.5422837 -75.0118652
4266
HALE AVE
40.6823346 -73.8804596
4267
BYRON AVENUE
40.8965999 -73.85359350000002
4268
S/W C/O LYDIG
40.8545668 -73.86654709999999
4269
I/O ARGYLE RD
40.6398746 -73.9660674
4270
New Lots Ave
40.6632877 -73.8898367
4271
SB LUTEN AV AMBOY RD
40.530871 -74.1939703
4272
32 ST
40.7484063 -73.9884498
4273
MCGRAW AVENUE
40.8345952 -73.8612096
4274
WEST 37 ST
40.754091 -73.99207899999999
4275
WEST 79TH STREET
40.7838194 -73.9799099
4276
WEST 128
40.8112596 -73.9476722
4277
Jackson Pl
40.6625221 -73.98634899999999
4

41.0188032 -73.6813591
4445
ROCHAMEBEAU AVE
40.8789563 -73.8802103
4446
Jesup Ave
40.8431946 -73.919252
4447
ELLERY ST
40.6989682 -73.944096
4448
Hudson Blvd East
40.7551587 -74.0005147
4449
GREENWICH STREET
40.7221373 -74.0097722
4450
E/B 141 ST
40.5744994 -73.8589645
4451
DIGNEY AVE
40.893406 -73.8515133
4452
EAST 215 STREET
40.8789178 -73.8585639
4453
MATILDA AVENUE
40.9013015 -73.85451499999999
4454
C/O BURNSIDE AV
40.8537474 -73.9122927
4455
E 48TH ST
40.7545714 -73.9716804
4456
E KINGBRIDGE
40.8649485 -73.89401610000002
4457
GRANT AVENUE
40.6830197 -73.8671366
4458
141 RD
40.6696207 -73.75326249999999
4459
KINGS PLAZA
40.610028 -73.9199938
4460
Walker Ct
40.5534737 -74.1779492
4461
Lyle Ct
40.56531409999999 -74.1383213
4462
W 237th St
40.8842146 -73.90053979999999
4463
C/O 202 ST
40.746127 -73.775296
4464
HULL STREET
40.6794667 -73.9115845
4465
DARKSIDE AVE
40.7127753 -74.0059728
4466
C/O 58 RD
40.7824262 -73.44047069999999
4467
69 AV
40.719373 -73.8506025
4468
84TH STREET
40.786

40.7592474 -73.9672198
4637
B 124 ST
40.5790654 -73.844732
4638
WOODWARD
40.7189134 -73.9914967
4639
C/O CASE ST
40.8054513 -73.91597569999999
4640
Remsen Pl
40.7222152 -73.89800450000001
4641
E 88TH STREET
40.7794483 -73.9514145
4642
EASTERN PKWY (W/B) @
40.6693118 -73.9366428
4643
S/W C/O 82 ST
40.77547819999999 -73.9540605
4644
32 STREET
40.7484063 -73.9884498
4645
Rushmore Ave
40.7635863 -73.7400809
4646
NOEL ROAD
40.6078337 -73.81835280000001
4647
W 6 ST
40.6030329 -73.97879879999999
4648
C/O QUEENS PLAZA SOU
40.750909 -73.94213599999999
4649
W 225 STREET
40.8742793 -73.90924729999999
4650
E/S LONGFELLOW AV
40.8228128 -73.8873246
4651
HAVEMAYER ST
40.712239 -73.95713599999999
4652
R/O 71 HESTER ST
40.7161122 -73.9909457
4653
CORNER OF 70TH ST
40.7772491 -73.9824378
4654
GRAY STREET
40.6251515 -74.081574
4655
ANTIN PL
40.8515466 -73.86488680000001
4656
R/S E34 ST
40.7508025 -73.98948349999999
4657
SCHENECTADY AVENUE
40.6479969 -73.9329114
4658
CRIMI RD
41.737565 -73.975296
4659
LIN

40.7095032 -74.00018469999999
4828
S/S WEST 31 ST
40.7495328 -73.9930084
4829
E152 ST
40.8156548 -73.90624389999999
4830
Sunnyside Ave
40.68138039999999 -73.8926429
4831
C/O LOUISE NINE BLVD
40.8318027 -73.89226080000002
4832
N/W C/O E 104
43.2994285 -74.21793260000001
4833
E 128 ST
40.8066553 -73.9368736
4834
BRIGHTON BEACN AVE
40.577555 -73.96142890000002
4835
Slater Blvd
40.58018879999999 -74.08876889999999
4836
MIDDLETON STREET
40.702236 -73.95226509999999
4837
N/W C/O 104 ST
40.7909485 -73.94723789999999
4838
FULTON WALK
40.5570922 -73.91763379999999
4839
Pearl St
40.7070738 -74.0062518
4840
WEST 28 STREET
40.7490938 -73.9969443
4841
BEACH 51 STREET
40.5798205 -74.0104523
4842
Avenue O
40.6121133 -73.9667857
4843
W 50th St
40.7626366 -73.9869403
4844
BOERUM STREET
40.70596889999999 -73.9422283
4845
PINEHURST
42.73222 -78.93527999999999
4846
N/E C/O LEONARD ST
40.7410236 -73.9917641
4847
65th Ave
40.7530342 -73.7394585
4848
E 213 ST
40.8848599 -73.8784735
4849
Goller Pl
40.6065288 

In [27]:
#add lat lon to precinct dataframe 15k-20k
street_lat_4 = dict()
for i in street_coords_4:
    street_lat_4[i]=street_coords_4[i][0]
street_lon_4 = dict()
for i in street_coords_4:
    street_lon_4[i]=street_coords_4[i][1]

In [28]:
#call get_lat_lon for street name 20k-25k
street_names_5 = all_street_names[20000:25000]
street_coords_5 = dict()
ctr=0
for i in street_names_5:
    ctr+=1
    print(ctr)
    try:
        street_coords_5[i] = get_lat_lon_street(i)
    except:
        street_coords_5[i] = "not valid"

1
C/O BASSETT
40.5920087 -73.9559032
2
WOODPOINT RD
40.7170591 -73.9417643
3
WALDERN ST
41.3416651 -73.6755742
4
SEASIDE COURTS
40.5743493 -73.9723759
5
DEWITT AVE
40.6561173 -73.89665750000002
6
W/S 153 ST
40.8297567 -73.9426914
7
Willow Ave
40.8023147 -73.91275759999999
8
Lott Pl
40.6226377 -73.9349482
9
E/S/O/N/O  CONKLIN A
42.0342437 -75.80380099999999
10
BEACH 100TH STREET
40.5845303 -73.82155139999999
11
C/O VARET ST
40.703798 -73.93657499999999
12
62ND DR
40.7343236 -73.8554623
13
B 125 ST
40.8180137 -73.9608451
14
E 43RD STREET
40.7520423 -73.97536040000001
15
E 48 ST
40.7545714 -73.9716804
16
23RD ST
40.7453063 -73.9984746
17
N/S SEABREEZE AVE
40.8188636 -72.66775009999999
18
69 AVENUE
40.719373 -73.8506025
19
Horace Harding Blvd
40.7453741 -73.77248209999999
20
45 AVENUE
40.7422611 -73.8759688
21
LAURELTON PKWY
40.6748277 -73.7337802
22
EB E149 ST. BROOK AV
40.814943 -73.9149655
23
ROBERT WARNER
40.7658442 -73.9680576
24
CROCHERON AVE
40.7628067 -73.7958046
25
65th St Transve

40.7437259 -73.89987579999999
204
SPENCER PL
40.6813019 -73.9544744
205
121st Ave
40.678616 -73.78423
206
Winfield Ave
40.59842 -74.08391
207
WEST 45TH ST
40.75957160000001 -73.9893629
208
C/O 150
40.7824262 -73.44047069999999
209
E/S ST MARYS ST
40.8090812 -73.9115819
210
E 179th St
40.8474742 -73.89134609999999
211
E 229 DR N
40.8870419 -73.84428079999999
212
N/W C/O 13 AVE
40.617137 -74.011122
213
S/S W 37 ST
40.754091 -73.99207899999999
214
Howard Pl
40.6587489 -73.98065869999999
215
SHORE PKWY
40.6203345 -73.8975419
216
E 197th St
40.86724299999999 -73.889748
217
Pitt St
40.7178007 -73.9827574
218
N/W C/O 134 ST
40.8163934 -73.94834829999999
219
HOLL ST
40.722012 -73.98799919999999
220
W/S INDEPENDENCE AVE
40.8953456 -73.9117203
221
N/E C/O E 96 ST
40.7851817 -73.9474171
222
Kane Pl
40.678222 -73.92369029999999
223
S/W C/O 60 AVE
40.7294026 -73.87524839999999
224
SB 14 AV 86 ST - CRO
40.62535159999999 -74.0400306
225
Terrace Veiw Ave
40.8772627 -73.9120373
226
W/S CHESTNUT AVE
40.

40.7127753 -74.0059728
399
ENGERT AV
40.7216496 -73.9450925
400
C/O 18 AVE
40.616585 -73.992334
401
N/O 136TH ST
40.817589 -73.9470202
402
CROMWELL AVE
40.5894292 -74.0945482
403
W 64th St
40.7735391 -73.9853266
404
Robinson Ave
40.5400853 -74.15412239999999
405
114 PL
40.6713748 -73.82547810000001
406
Storer Ave
40.5390227 -74.2348102
407
CEDARWOOD RD
43.212169 -77.60796309999999
408
PARK A VE
40.8043256 -73.939393
409
COLLEGE PL
40.6970217 -73.99410759999999
410
SIDE C/O 462 E 38 ST
40.7455902 -73.97064979999999
411
WIN
40.7483698 -73.989748
412
150th Ave
40.657107 -73.7669873
413
Hendricks Ave
40.6403063 -74.08435419999999
414
E 97 TH
40.7861708 -73.9495271
415
Great Jones Aly
40.7269457 -73.9947344
416
PAERDEGAT 14
40.6282301 -73.901551
417
TERMINAL 2 OUTER RD
40.7527262 -73.9772294
418
E 33rd St
40.7455581 -73.97980319999999
419
N/S KINGS HWY
40.6050602 -73.9805536
420
WALTON ST
40.7021677 -73.9493866
421
SICKLES ST
40.86132569999999 -73.927793
422
EDGECOMBE AVE
40.8288583 -73.940

40.8799237 -73.8978628
600
E 96th St
40.7853417 -73.9496916
601
S/S BOOME AVE
40.68745699999999 -73.365439
602
19th St
40.7370595 -73.98653360000002
603
E 229 D/S
40.8882314 -73.85137739999999
604
W 159TH STREET
40.8342199 -73.9413886
605
NB OCEAN PKWY @ AVE
40.61287799999999 -73.96838620000001
606
W 230 ST
40.8789989 -73.9090392
607
S/E C/O 6THA VE
40.74540349999999 -73.9923075
608
EXTERIOR ST
40.8198959 -73.9305586
609
AMSTERDAM AVE (N/B)
40.8054543 -73.96247869999999
610
47th St
40.7607273 -73.9882141
611
Bristow St
40.8328669 -73.8950704
612
E 117 STREET
40.79768019999999 -73.93728949999999
613
W/S 17 AVE
40.6183313 -73.9947535
614
CLOVE ROAD
40.6176306 -74.10455080000001
615
C/O TILLARY ST
40.6951965 -73.9952199
616
175th St
40.8454678 -73.936636
617
E 162 ST
40.8249578 -73.9139615
618
OTTO RD
40.704971 -73.8898402
619
SB AMSTERDAM AV W148
40.8134463 -73.9562105
620
CARNARSIE PIER
40.62869060000001 -73.8839151
621
Ilyse Ct
40.5651519 -74.139725
622
COMMON AVE
40.8343859 -73.868855

40.642753 -73.9002904
796
80th Rd
40.7266118 -73.78634939999999
797
C/O N/E CONEY ISLAND
40.5745189 -73.97854629999999
798
E 166TH ST
40.8287869 -73.9102985
799
E/S SHERMAN AVE
40.8643666 -73.9237512
800
RIIS PARK LOT B4
40.5670695 -73.87177020000001
801
S/S 58 ST
40.6340424 -74.0040673
802
R/S 90TH
40.7899188 -73.97238159999999
803
Colden St
40.7502563 -73.82277400000001
804
Review Ave
40.73317429999999 -73.9368349
805
W 9
40.67509080000001 -73.9975357
806
E.128TH ST.
40.8066553 -73.9368736
807
SETON PL
40.6299887 -73.9750123
808
EXTERIOR AVE
40.8198959 -73.9305586
809
SUMPTER STREET
40.6803198 -73.917182
810
SB FARMERS BV. 144 A
34.4208305 -119.6981901
811
Lorraine Ave
40.9151411 -73.8178819
812
COLDEN ST
40.7502563 -73.82277400000001
813
E 14TH STREET
40.7313474 -73.9825664
814
B 145 ST
40.8242227 -73.94519079999999
815
DRUMGOOLE EAST
40.54033190000001 -74.185794
816
E 94 ST
40.78433210000001 -73.95131860000001
817
Darcey Ave
40.6082922 -74.1508757
818
S 5th Pl
40.7105971 -73.960860

...

40.8014193 -73.9581317
4070
W 247 ST
40.895169 -73.9119253
4071
EAST 165 STREET
40.8264916 -73.90768280000002
4072
Prospect Park South
40.6457741 -73.96610609999999
4073
N/E C/O W 174 ST
40.8448371 -73.93709679999999
4074
S/S SHORE AVE
40.692151 -73.802463
4075
C/O BALCOM
43.2994285 -74.21793260000001
4076
MOSHOLU PKW S
40.873893 -73.8833964
4077
WEST 122ND ST
40.808865 -73.954048
4078
EAST 41
40.7501902 -73.9751125
4079
COLUMBIA ST
40.7171876 -73.9801753
4080
EAST END ST
40.7736925 -73.9456902
4081
S/W QUEENS BLVD
40.7340424 -73.8730341
4082
N 5TH ST
40.7167367 -73.9589105
4083
NB PARSONS BV. 82 DR
40.7168083 -73.8081038
4084
GASTERN PKWY
40.6693118 -73.9366428
4085
RIDGE AVE
40.5997697 -74.0987629
4086
Home Lawn St
40.715696 -73.79268929999999
4087
W/S WHITTIER ST
40.8137179 -73.88340529999999
4088
88TH ST
40.7794483 -73.9514145
4089
HARRISON ST
40.7188367 -74.0106441
4090
SB RICHMOND RD. RALP
40.58506819999999 -74.10630069999999
4091
SB 4 AVE. 5 ST- 11 S
40.7278477 -73.9910613
4092


40.7657244 -73.8918474
4263
Fox Ter
40.8725349 -73.8444584
4264
S/S WEST 39TH ST
40.7552425 -73.99096949999999
4265
DUPONT STREET
40.7358706 -73.9551875
4266
214th St
40.7303436 -73.7553908
4267
CLOVER PLACE
40.7182925 -73.76863
4268
6TH STREET
40.7251787 -73.9833058
4269
WEST 147 ST
40.8253061 -73.9437651
4270
R/S CX BX EXP
40.7127753 -74.0059728
4271
SPARTON AVENUE
43.2309919 -77.99849300000001
4272
N/E C/O ARGYLE RD
40.7410236 -73.9917641
4273
C/O WASHINGTON ST
40.7397259 -74.008496
4274
BENSON
43.2520154 -74.2784691
4275
EAST 208TH STREET
40.87910129999999 -73.88204449999999
4276
GAFTON ST
40.6643875 -73.9181498
4277
E 133rd St
40.8014949 -73.9148865
4278
MAGUIRE AV
40.5353378 -74.2116787
4279
JAM
40.7467438 -73.9907644
4280
RANDALLSS ISLAND WAR
40.7932271 -73.92128579999999
4281
Oak Ave
40.7503423 -73.8100499
4282
W 42ND ST
40.7578775 -73.99132759999999
4283
C/O 33 ST / 10 AVE
40.7478955 -73.9858445
4284
Hartford Ave
40.6239136 -74.109725
4285
7 ST AVE
40.7260554 -73.9835429
4286


40.6305129 -73.9281689
4453
Larue Ave
40.740825 -73.857097
4454
FIRST ST
40.7236185 -73.9892192
4455
S/S OLD SLIP
40.7036786 -74.0085142
4456
NB ADAM CLAYTN PWL J
40.8102439 -73.9479964
4457
Amador St
40.6235442 -74.16472519999999
4458
C/O B 81 ST
40.7841488 -73.9765937
4459
51 ST ST
40.7632683 -73.9864745
4460
Westgate St
40.6735373 -73.7612362
4461
CORPORAL STONE ST
40.766784 -73.7712494
4462
COLONIAL ROAD
40.6307554 -74.0339813
4463
C/O BEDFORD AVE
40.6809646 -73.9532796
4464
97 AVE
40.6852359 -73.84937479999999
4465
BLEECKER  ST
40.7288134 -74.00043099999999
4466
44TH AVENUE
40.7589526 -73.98982560000002
4467
93RD STREET
40.783266 -73.9507404
4468
E 45 STREET
40.7530453 -73.9738796
4469
WAINWRIGHT AV
40.5524452 -74.1701488
4470
Rockwood St
40.8419074 -73.9131913
4471
N/S E 23RD ST
40.7453063 -73.9984746
4472
WEST 144 ST
40.8226338 -73.94351
4473
BEACH 19TH
40.5985434 -73.7527454
4474
NOSTRAN AVE
40.6414346 -73.9485729
4475
MAUJER STREET
40.7109627 -73.9431346
4476
Chappell St
40.63

40.7032775 -74.0170279
4643
W 88TH ST
40.7890023 -73.9740865
4644
C/O 1542 WATSON AVE
40.8260596 -73.87737369999999
4645
EAST 87 ST
40.778951 -73.9521984
4646
Park Pl West
41.0361476 -73.7514211
4647
Northern Blvd
40.76539950000001 -73.8175923
4648
IRVING AVENUE
40.6988381 -73.91407199999999
4649
W 110
40.8009929 -73.9590858
4650
W 154th St
40.8289977 -73.9389727
4651
CORNELIA ST
40.731479 -74.002015
4652
Bronxdale Ave
40.8474697 -73.8599132
4653
WEST 22 ST
40.7446311 -73.9989763
4654
C/O REGENT
40.7517024 -73.9897146
4655
E 55TH ST
40.7587698 -73.96802749999999
4656
W 17
40.7413184 -74.0002966
4657
58th Rd
40.7205218 -73.9052396
4658
BENCHELEY PL
40.8740302 -73.825924
4659
WHITNEY PL
40.5978774 -73.9722901
4660
E 147 ST
40.8119081 -73.9101925
4661
C/O DECATUR
42.5922968 -74.7512261
4662
Princeton Ave
40.56813959999999 -74.118482
4663
C/O ROOSEVELT AVE
40.7458626 -73.89966299999999
4664
N/S DENRELY PL
40.7127753 -74.0059728
4665
EVERGREEN ST
40.5574833 -74.1417542
4666
S/S STANLEY  AVE

40.8096973 -73.91449209999999
4836
NB OCEAN PKWY @ 18TH
40.6331359 -73.9721071
4837
C/O TAYLOR AVE
40.84162800000001 -73.867892
4838
Eltingville Blvd
40.5516885 -74.1658837
4839
WILLIAMBURG ST
40.7016756 -73.9614503
4840
C/O 105 AVE
40.6960881 -73.8031985
4841
Merrill St
40.6837741 -73.7713368
4842
W 238th St
40.886714 -73.90348569999999
4843
Rae Ave
40.5357905 -74.1738807
4844
WEST 251 ST
40.8957623 -73.8973448
4845
SUBURBAN PL
40.8359804 -73.8914163
4846
ALLENTON AVE
40.865412 -73.8519496
4847
Kosciusko St
40.6918106 -73.9409927
4848
80TH ROAD
40.7266118 -73.78634939999999
4849
PENN ST
40.750568 -73.993519
4850
PARK STREET
40.5698396 -74.13203690000002
4851
150TH RD
40.6693192 -73.8404855
4852
89th St
40.7800581 -73.95092720000001
4853
Albert Rd
40.6702834 -73.84128969999999
4854
JULES ST
40.7280896 -73.9861248
4855
EAST 35 STREET
40.7464842 -73.97784399999999
4856
GOTHIC DR
40.71380509999999 -73.7956131
4857
CENTRE STREET
40.7163828 -74.0012118
4858
BRIGHTON 11 STREET
40.580222 -73.

In [29]:
#add lat lon to precinct dataframe 20k-25k
street_lat_5 = dict()
for i in street_coords_5:
    street_lat_5[i]=street_coords_5[i][0]
street_lon_5 = dict()
for i in street_coords_5:
    street_lon_5[i]=street_coords_5[i][1]

In [30]:
#call get_lat_lon for street name 25k-max
street_names_6 = all_street_names[25000:26227]
street_coords_6 = dict()
ctr=0
for i in street_names_6:
    ctr+=1
    print(ctr)
    try:
        street_coords_6[i] = get_lat_lon_street(i)
    except:
        street_coords_6[i] = "not valid"

1
W 72 ST
40.7785351 -73.98145819999999
2
E 163RD STREET
40.8235775 -73.9067532
3
WEST FORT HAMILTON
40.6094454 -74.02854339999999
4
86 AVENUE
40.60934659999999 -74.0060919
5
GERRITSEN  AVE
40.5984539 -73.9314459
6
57 STREET
40.767267 -73.9841435
7
ST JOHNS PLACE
40.6715288 -73.9475595
8
C/O E 31 ST
40.7688417 -73.9195183
9
WEST 51 ST
40.7632683 -73.9864745
10
BEACH 95 ST
40.5856599 -73.8172822
11
EAST 172ND STREET
40.8312664 -73.8788457
12
AMSTERDAM AVENUE
40.8134463 -73.9562105
13
VANDERBILT AVENUE
40.7540445 -73.9772117
14
Coale Ave
40.6134458 -74.1281833
15
EB HILLSIDE AVE @ PA
40.7185003 -73.7652669
16
E 102nd St
40.7894475 -73.947576
17
E 234th St
40.89537410000001 -73.8606994
18
N/W/C BRIGHTON 7 ST
40.5797668 -73.9613078
19
Elton Ave
40.821813 -73.91322339999999
20
C/O 131 ST
40.808197 -73.9352993
21
69th Ln
40.7235853 -73.8912819
22
Convent Ave
40.8211769 -73.9487784
23
148 ROAD
40.6544803 -73.7367656
24
Sheraden Ave
40.6074418 -74.13802199999999
25
Apex Pl
40.7353032 -73.85215

40.7578775 -73.99132759999999
202
Brighton 1st Pl
40.5767088 -73.9659964
203
WALES AVE
40.8114057 -73.9077786
204
GILDER SLEEVE
40.8094782 -73.85424309999999
205
N/E TILLOTSON AVE
40.7127753 -74.0059728
206
W 2ND ST
40.5867128 -73.97139659999999
207
LGA LOT P10
40.777027 -73.8834376
208
W/S LURTING AVE
40.8502147 -73.85466149999999
209
MERRICK BLVD
40.6837389 -73.769544
210
REDFORD
44.6083776 -73.8043096
211
Monaco Pl
40.6768114 -73.9090403
212
R/O 2275 RANDALL AVE
40.819731 -73.8457391
213
E/S SOUTH ST
40.7090604 -73.99727419999999
214
Kermit Pl
40.648489 -73.97267200000002
215
SB WEBSTER AV. PARKS
40.8622831 -73.890278
216
W 51 ST
40.7632683 -73.9864745
217
E 164TH ST
40.8264153 -73.9117938
218
CENTRAL PARK WEST
40.784612 -73.9698224
219
NEW LOTS AVENUE
40.6632877 -73.8898367
220
Doone Ct
40.5903982 -73.958635
221
6TH  AVE
40.742903 -73.99279779999999
222
DICKENS ST
40.6055257 -73.7608806
223
WHITE AVE
40.61008899999999 -74.0292838
224
S/E C/O E 151 ST
40.8280385 -73.9424401
225
Nevi

40.5609673 -74.12861699999999
401
Mt. Olivet Cres
40.7155902 -73.8968825
402
NB BEDFRD AVE. LEXNG
40.6885189 -73.9437122
403
N/W 77 ST
40.7814934 -73.97845199999999
404
WT PLAINS RD
40.8563009 -73.8676435
405
W/S 97 ST
40.7944762 -73.9692453
406
STERLING AVE
40.5692751 -74.1140825
407
AVE PL
40.7605465 -73.9727699
408
E 51 STREET
40.7564635 -73.97032229999999
409
C/O BEEKMAN AV
41.084349 -73.8624987
410
Spritz Rd
40.672777 -73.8469624
411
N 36 ST
40.75347259999999 -73.9925244
412
Central Park South
40.7661851 -73.9773038
413
COURTLANDT AVE
40.8206448 -73.9177149
414
HALLISTER ST
40.5259881 -74.2186009
415
QUAY STREET
40.7262035 -73.95860789999999
416
C/O TOMPKINS CT
40.6398639 -74.120413
417
EAST 3 STREET
40.7236548 -73.9852944
418
Hannah St
40.635937 -74.0759033
419
20TH AE
40.7127753 -74.0059728
420
ACADEMY
40.7051704 -74.0159694
421
N/S MOFFAT ST
40.6874594 -73.90648159999999
422
CASANOUA
40.734128 -73.951962
423
E 126 ST
40.8047954 -73.9363171
424
49 ST
40.76203659999999 -73.987456

40.7400757 -73.9856097
598
N/E C/O E 26 ST
40.7475915 -73.9980082
599
Avenue V
40.597274 -73.9509515
600
W 14TH STREET
40.7393254 -74.00148089999999
601
149 PL
40.7639136 -73.8155137
602
WASHINGTON AVE
40.84722989999999 -73.9397819
603
St Austins Pl
40.6363288 -74.10805789999999
604
139th St
40.8196371 -73.9460725
605
Arlington Ave
40.9008066 -73.9083891
606
HUBBARD ST
40.5880789 -73.9631681
607
C/O MARION
43.1632559 -77.146153
608
Amherst Ave
40.558211 -74.116441
609
Thayer St
40.8632456 -73.9269309
610
1ST PL
40.6806208 -73.9979808
611
Library Ave
40.8461484 -73.8183371
612
E 97 STREET
40.7861708 -73.9495271
613
E/S STEBBINS AVE
40.83329 -73.8941962
614
MICKLE AVENUE
40.8697672 -73.8415662
615
DWIGHT ST
40.67588 -74.0092167
616
120 AVE
40.684997 -73.766969
617
AVE OF AMERICAS
40.742903 -73.99279779999999
618
BUTLER PL
40.6751207 -73.9682096
619
STEVENSON HIGH SCHOO
40.8215141 -73.85541409999999
620
Dalton Ave
40.5726479 -74.12698689999999
621
W 151 ST
40.8280385 -73.9424401
622
S/E E

40.7068352 -73.92141
796
N/W C/O STATION RD
43.2994285 -74.21793260000001
797
WOLCOTT STREET
40.6778416 -74.0136668
798
WALKER
40.7302 -73.991252
799
O/O 160 E 3
40.7233361 -73.98476029999999
800
MANOR AV
40.8289769 -73.87640979999999
801
SB AMSTERDAM AV. W15
40.8134463 -73.9562105
802
Brielle Ave
40.5955348 -74.1343957
803
C/O CASTLEHILL AVE
40.8367227 -73.8520364
804
58th Dr
40.7197195 -73.9089052
805
EAST 84 ST
40.7772689 -73.9543839
806
39 AVENUE
40.7543504 -73.93600649999999
807
E 47 ST
40.7543987 -73.97321250000002
808
EAST MOUNT EDEN AVE
40.8431083 -73.9104298
809
W 207th St
40.8658136 -73.9199866
810
EAST TREMONT AVENUE
40.8410341 -73.8644818
811
S/S STANTON
40.7209979 -73.98694929999999
812
WEST 48
40.7613861 -73.9878365
813
31ST RD
40.7659917 -73.9291606
814
Shale St
40.5839373 -74.1507578
815
EDGEWATER ROAD
40.8194145 -73.8839106
816
SB WEBSTER AVE@ E TR
40.8647524 -73.8875207
817
Ellwood St
40.8604107 -73.928534
818
WEST 5TH
40.578466 -73.9722652
819
I/O 865 COLUMBUS AVE
40

40.6453071 -73.9579844
994
S/E C/O LENOX
43.05833 -75.70472
995
Croft Ct
40.5627237 -74.13974259999999
996
E MOSHOLU S
40.873893 -73.8833964
997
152 ST
40.8287195 -73.9421796
998
GRAVESEND NECKS RD
40.5945792 -73.959467
999
B 87TH
40.7883666 -73.9745459
1000
BEECH AVE
40.758091 -73.816786
1001
STEPHENS ST
40.7737414 -73.9504685
1002
KETCHAM STREET
40.7446663 -73.8830115
1003
CORNER OF E 134 ST
40.8086434 -73.9451464
1004
Piave Ave
40.5947123 -74.070264
1005
177th Pl
40.6928467 -73.77159309999999
1006
EAST 51TH ST
40.7564635 -73.97032229999999
1007
BAY 17 ST
40.604338 -74.0069121
1008
S/N/C 12 ST
40.7300131 -73.9835216
1009
E/S LEXINGTON SOUTH
40.773008 -73.96032
1010
E 155th St
40.82003210000001 -73.9153412
1011
257 ST
40.7367448 -73.71100899999999
1012
C/O ROBIN RD
43.0183655 -78.77080409999999
1013
W/S OF ST NICHOLAS T
40.7882783 -73.9540925
1014
C/O BANHER
40.7392791 -73.9887999
1015
CORNER OF W 197
40.8807333 -73.8838627
1016
Riverdale Ave
40.8844368 -73.90762699999999
1017
S/E COR

40.7163828 -74.0012118
1184
Lovell Ave
40.5914212 -74.1497657
1185
Graves St
40.6021945 -74.1294667
1186
MOFFATT STREET
42.4548704 -75.0874401
1187
AMTERDAM AVE
40.8134463 -73.9562105
1188
W BURNSIDE AV
40.8537474 -73.9122927
1189
SB SOUTHRN BV. BARET
40.7127753 -74.0059728
1190
E 159 STREET
40.8233969 -73.9160033
1191
WB PARK AVE @ WAVERL
42.0012795 -76.5384519
1192
Port Richman Avenue
40.6325043 -74.1374251
1193
Greaves Ave
40.5591863 -74.14808409999999
1194
BRYANT ST
40.6689703 -74.00336829999999
1195
Kimberly Pl
40.8778054 -73.9058798
1196
THOMAS BOYLAND AVE
40.6688509 -73.91347309999999
1197
LONG PLACE NORTH
40.7127753 -74.0059728
1198
174TH ST
40.8448371 -73.93709679999999
1199
WEST 136TH
40.817589 -73.9470202
1200
Ryan Pl
40.527385 -74.1665586
1201
Clarke Pl
40.8376763 -73.9172186
1202
KINGBRIDGE ROAD
40.8649485 -73.89401610000002
1203
BEDFPRD AVE
40.6537806 -73.956165
1204
W 45 STREET
40.75957160000001 -73.9893629
1205
R/S 55 AVE
40.7371917 -73.87109459999999
1206
C/O CRABTREE 

In [31]:
#add lat lon to precinct dataframe 25k-max
street_lat_6 = dict()
for i in street_coords_6:
    street_lat_6[i]=street_coords_6[i][0]
street_lon_6 = dict()
for i in street_coords_6:
    street_lon_6[i]=street_coords_6[i][1]

In [32]:
street_lat.update(street_lat_2)
street_lat.update(street_lat_3)
street_lat.update(street_lat_4)
street_lat.update(street_lat_5)
street_lat.update(street_lat_6)
print(len(street_lat))

26227


In [33]:
street_lon.update(street_lon_2)
street_lon.update(street_lon_3)
street_lon.update(street_lon_4)
street_lon.update(street_lon_5)
street_lon.update(street_lon_6)
print(len(street_lon))

26227


In [34]:
df_street['Latitude']= df_street['Street Name'].map(street_lat)
df_street['Longitude']= df_street['Street Name'].map(street_lon)

In [35]:
df_street['count']=1
print(df_street.head())
df_street.to_csv("street.csv")

            Street Name Latitude Longitude  count
0           REGIS DRIVE  40.6199  -74.1594      1
1              Broadway  40.8124  -73.9605      1
2            E 187TH ST  40.8568  -73.8899      1
3  SB BAILEY AVE @ W 22  40.5911  -73.9832      1
4             E 17th St  40.7356  -73.9866      1
